In [1]:
import polars as pl

In [2]:
# Read processed data
df = pl.read_csv("../data/processed/day-01.csv")

In [3]:
# Calculate the distance between the left and right columns
df = df.with_columns((pl.col('left') - pl.col('right')).abs().alias('distance'))

In [4]:
# Calculate the sum of the distances
df.select(pl.col('distance').sum())

distance
i64
2756096


In [5]:
# Count occurrences in right column
counts = df.group_by('right').agg(
    pl.col('right').count().alias('frequency')
)

# Join the counts back to the original DataFrame
df = df.select('*').join(
    counts, 
    left_on='left', 
    right_on='right', 
    how='left'
)

# Fill null values with 0
df = df.with_columns(pl.col('frequency').fill_null(0))

# Calculate similarity score
df = df.with_columns(
    (pl.col('left') * pl.col('frequency')).alias('multiplication'),
)

In [6]:
# Calculate the sum of the similarity scores
df.select(pl.col('multiplication').sum())

multiplication
i64
23117829
